In [87]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from behav_analysis import Data_Functions, Participant_Behav
    
class Participant_Watch():
    def __init__(self, par_num):
        self.data_fun = Data_Functions()
        self.par_num = par_num
        self.par_ID = f"participant_{self.par_num}"
        data_dir = r"C:\Kernel\participants"
        #self.par_dir = os.path.join(os.getcwd(), "participants", self.par_ID)
        self.par_dir = os.path.join(data_dir, self.par_ID)
        self.par_behav = Participant_Behav(par_num=self.par_num)
        self.exp_order = self.par_behav.exp_order
        self._dir_list = self._get_data_dirs()
        
        self.modalities = ["ACC", "BVP", "EDA", "HR", "IBI", "TEMP"]
        self.modality_df_dict = self._create_modality_df_dict()
        self.marker_ts_df = self.par_behav.marker_ts_df
        self.exp_modality_dict = self._create_exp_modality_dict()

    def _get_data_dirs(self):
        watch_dir = os.path.join(self.par_dir, "watch_data")
        dir_list = []
        for dir_name in os.listdir(watch_dir):
            dir_list.append(os.path.join(watch_dir, dir_name))

        return dir_list

    def _create_modality_df(self, modality):
        df_list = []
        for watch_dir in self._dir_list:
            filepath = os.path.join(watch_dir, modality + ".csv")
            temp_df = pd.read_csv(filepath)
            initial_ts = int(float(temp_df.columns[0]))
            if modality != "IBI":
                samp_freq = int(temp_df.iloc[0][0])
                ts_col = pd.Series([initial_ts + i/samp_freq for i in range(temp_df.size)])
                dt_col = pd.Series([datetime.datetime.fromtimestamp(ts) for ts in ts_col])
                temp_df = temp_df[1:]
                temp_df.insert(loc=0, column="timestamps", value=ts_col)
                temp_df.insert(loc=1, column="datetime", value=dt_col)

                if modality == "ACC":
                    temp_df.rename(columns={temp_df.columns[2]: "accel_x", temp_df.columns[3]: "accel_y", temp_df.columns[4]: "accel_z"}, inplace=True)
                    temp_df["accel_x"] = temp_df["accel_x"]/64
                    temp_df["accel_y"] = temp_df["accel_y"]/64
                    temp_df["accel_z"] = temp_df["accel_z"]/64
                elif modality == "BVP" or modality == "EDA" or modality == "HR":
                    temp_df.rename(columns={temp_df.columns[2]: modality}, inplace=True)    
                elif modality == "TEMP":
                    temp_df.rename(columns={temp_df.columns[2]: "TEMP_C"}, inplace=True)
                    temp_F_col = pd.Series([self.data_fun.c_to_f(temp_C) for temp_C in temp_df["TEMP_C"]])
                    temp_df = temp_df.drop(columns="TEMP_C")
                    temp_df.insert(loc=2, column="TEMP", value=temp_F_col)
            elif modality == "IBI":
                ts_col = temp_df.iloc[:, 0] + initial_ts
                dt_col = pd.Series([datetime.datetime.fromtimestamp(ts) for ts in ts_col])
                temp_df.insert(loc=0, column="timestamps", value=ts_col)
                temp_df.insert(loc=1, column="datetime", value=dt_col)
                temp_df = temp_df.drop(columns=temp_df.columns[2])
                temp_df.rename(columns={temp_df.columns[2]: modality}, inplace=True)    
            df_list.append(temp_df)
        df = pd.concat(df_list, axis=0)
        df.reset_index(inplace=True, drop=True)

        return df

    def _create_modality_df_dict(self):
        modality_df_dict = {}
        for modality in self.modalities:
            modality_df_dict[modality] = self._create_modality_df(modality)
        
        return modality_df_dict

    def _create_exp_modality_dict(self):
        def _get_block_col(exp_name, df):  
            exp = self.par_behav.get_exp(exp_name=exp_name)
            num_blocks = exp.num_blocks
            num_rows = df.shape[0]
            df = df.reset_index()
            col_list = []
            
            if exp_name == "audio_narrative" or exp_name == "video_narrative_cmiyc" or exp_name == "video_narrative_sherlock":
                for ts_tuple, block in self.par_behav.by_block_ts_df[exp_name].items():
                    start_ts = ts_tuple[0]
                    end_ts =  ts_tuple[1]
                    block_start_idx = self.data_fun.get_start_index_ts(df, start_ts)
                    block_end_idx = self.data_fun.get_end_index_ts(df, end_ts)
                    if block_start_idx == None or block_end_idx == None:
                        pass
                    else:
                        col_list.append(self.data_fun.create_col(None, block_start_idx))
                        col_list.append(self.data_fun.create_col(block, block_end_idx-block_start_idx))
                        col_list.append(self.data_fun.create_col(None, num_rows-block_end_idx))
            elif exp_name == "go_no_go" or exp_name == "king_devick" or exp_name == "n_back" or exp_name == "resting_state" or exp_name == "tower_of_london" or exp_name == "vSAT":
                for i, (ts_tuple, block) in enumerate(self.par_behav.by_block_ts_df[exp_name].items()):
                    start_ts = ts_tuple[0]
                    end_ts =  ts_tuple[1]
                    block_start_idx = self.data_fun.get_start_index_ts(df, start_ts)
                    block_end_idx = self.data_fun.get_end_index_ts(df, end_ts)
                    if block_start_idx == None or block_end_idx == None:
                        pass
                    else:
                        if i == 0:
                            col_list.append(self.data_fun.create_col(None, block_start_idx))
                            col_list.append(self.data_fun.create_col(block, block_end_idx-block_start_idx))
                        elif i == exp.num_blocks-1:
                            col_list.append(self.data_fun.create_col(block, block_end_idx-block_start_idx))
                            col_list.append(self.data_fun.create_col(None, num_rows-block_end_idx))
                        else:
                            col_list.append(self.data_fun.create_col(block, block_end_idx-block_start_idx))

            block_col = pd.concat(col_list, axis=0, ignore_index=True)            
            
            return block_col

        def _get_trial_col(exp_name, df):  
            exp = self.par_behav.get_exp(exp_name=exp_name)
            num_blocks = exp.num_blocks
            num_rows = df.shape[0]
            df = df.reset_index()
            col_list = []
            
            if exp_name == "audio_narrative" or exp_name == "video_narrative_cmiyc" or exp_name == "video_narrative_sherlock":
                for i, (ts_tuple, block) in enumerate(self.par_behav.by_block_ts_df[exp_name].items()):
                    start_ts = ts_tuple[0]
                    end_ts =  ts_tuple[1]
                    block_start_idx = self.data_fun.get_start_index_ts(df, start_ts)
                    block_end_idx = self.data_fun.get_end_index_ts(df, end_ts)
                    if block_start_idx == None or block_end_idx == None:
                        pass
                    else:
                        col_list.append(self.data_fun.create_col(None, block_start_idx))
                        col_list.append(self.data_fun.create_col(f"{i+1}", block_end_idx-block_start_idx))
                        col_list.append(self.data_fun.create_col(None, num_rows-block_end_idx))
            elif exp_name == "go_no_go" or exp_name == "king_devick" or exp_name == "n_back" or exp_name == "resting_state" or exp_name == "tower_of_london" or exp_name == "vSAT":
                for i, (ts_tuple, block) in enumerate(self.par_behav.by_block_ts_df[exp_name].items()):
                    start_ts = ts_tuple[0]
                    end_ts =  ts_tuple[1]
                    block_start_idx = self.data_fun.get_start_index_ts(df, start_ts)
                    block_end_idx = self.data_fun.get_end_index_ts(df, end_ts)
                    if block_start_idx == None or block_end_idx == None:
                        pass
                    else:
                        if i == 0:
                            col_list.append(self.data_fun.create_col(None, block_start_idx))
                            col_list.append(self.data_fun.create_col(f"{i+1}", block_end_idx-block_start_idx))
                        elif i == exp.num_blocks-1:
                            col_list.append(self.data_fun.create_col(f"{i+1}", block_end_idx-block_start_idx))
                            col_list.append(self.data_fun.create_col(None, num_rows-block_end_idx))
                        else:
                            col_list.append(self.data_fun.create_col(f"{i+1}", block_end_idx-block_start_idx))

            trial_col = pd.concat(col_list, axis=0, ignore_index=True)            
            
            return trial_col

        exp_modality_dict = {}
        for exp_name in self.exp_order:
            start_dt, end_dt = self.data_fun.get_exp_dt(self.marker_ts_df, exp_name=exp_name)
            exp_modality_data_dict = {} 
            for modality, df in self.modality_df_dict.items():
                start_idx = self.data_fun.get_start_index_dt(df=df, start_dt=start_dt)
                end_idx = self.data_fun.get_end_index_dt(df=df, end_dt=end_dt)
                if start_idx == None or end_idx == None:
                    exp_modality_data_dict[modality] = None
                else:
                    modality_df = self.modality_df_dict[modality].iloc[start_idx:end_idx]
                    block_col = _get_block_col(exp_name=exp_name, df=modality_df)
                    modality_df = modality_df.reset_index()
                    modality_df.insert(0, "block", block_col)
                    trial_col = _get_trial_col(exp_name=exp_name, df=modality_df)
                    modality_df.insert(0, "trial", trial_col)
                    exp_modality_data_dict[modality] = modality_df
            exp_modality_dict[exp_name] = exp_modality_data_dict

        return exp_modality_dict

    def _plot_exp_regions(self, ax):
        for exp_name in self.exp_order:
            start_dt, end_dt = self.data_fun.get_exp_dt(self.marker_ts_df, exp_name=exp_name)
            ax.axvline(start_dt, linestyle="dashed", color="k", alpha=0.75)
            ax.axvline(end_dt, linestyle="dashed", color="k", alpha=0.75)
            if exp_name == "audio_narrative":
                ax.axvspan(start_dt, end_dt, color="yellow", alpha=0.4, label="Audio Narrative")
            elif exp_name == "go_no_go":
                ax.axvspan(start_dt, end_dt, color="green", alpha=0.4, label="Go//No-Go")
            elif exp_name == "king_devick":
                ax.axvspan(start_dt, end_dt, color="blue", alpha=0.4, label="King Devick")
            elif exp_name == "n_back":
                ax.axvspan(start_dt, end_dt, color="purple", alpha=0.4, label="N-back")
            elif exp_name == "resting_state":
                ax.axvspan(start_dt, end_dt, color="pink", alpha=0.4, label="Resting State")
            elif exp_name == "tower_of_london":
                ax.axvspan(start_dt, end_dt, color="orange", alpha=0.4, label="Tower of London")
            elif exp_name == "video_narrative_cmiyc":
                ax.axvspan(start_dt, end_dt, color="red", alpha=0.4, label="Video Narrative CMIYC")
            elif exp_name == "video_narrative_sherlock":
                ax.axvspan(start_dt, end_dt, color="olive", alpha=0.4, label="Video Narrative Sherlock")
            elif exp_name == "vSAT":
                ax.axvspan(start_dt, end_dt, color="cyan", alpha=0.4, label="vSAT")

    def plot_modality(self, modality):
        datetime_fmt = mdates.DateFormatter('%H:%M:%S')
        modality_df = self.modality_df_dict[modality]
        fig, ax = plt.subplots(1, 1, figsize=(15, 6))
        
        if modality == "ACC":
            ax.plot(modality_df["datetime"], modality_df["accel_x"], color="black")
            ax.plot(modality_df["datetime"], modality_df["accel_y"], color="darkslategray")
            ax.plot(modality_df["datetime"], modality_df["accel_z"], color="darkblue")
            ax.set_ylabel("Acceleration (g)", fontsize=16, color="k")
            ax.set_title("Acceleration", fontsize=20, color="k")
        elif modality == "BVP":
            ax.plot(modality_df["datetime"], modality_df["BVP"], color="k")
            ax.set_ylabel("BVP", fontsize=16, color="k")
            ax.set_title("Photoplethysmograph", fontsize=20, color="k")
        elif modality == "EDA":
            ax.plot(modality_df["datetime"], modality_df["EDA"], color="k")
            ax.set_ylabel("EDA (μS)", fontsize=16, color="k")
            ax.set_title("Electrodermal", fontsize=20, color="k")
        elif modality == "HR":
            ax.plot(modality_df["datetime"], modality_df["HR"], color="k")
            ax.set_ylabel("Heart Rate (BPM)", fontsize=16, color="k")
            ax.set_title("Heart Rate", fontsize=20, color="k")
        elif modality == "IBI":
            ax.plot(modality_df["datetime"], modality_df["IBI"], color="k")
            ax.set_ylabel("Interbeat Interval (seconds)", fontsize=16, color="k")
            ax.set_title("Heart Rate Variability", fontsize=20, color="k")
        elif modality == "TEMP":
            ax.plot(modality_df["datetime"], modality_df["TEMP"], color="k")
            ax.set_ylabel("Temperature (F)", fontsize=16, color="k")
            ax.set_title("Temperature", fontsize=20, color="k")

        ax.set_xlabel("Time", fontsize=16, color="k")
        ax.xaxis.set_major_formatter(datetime_fmt)
        self._plot_exp_regions(ax=ax)
        ax.legend(bbox_to_anchor=(1.0, 0.75), facecolor='white', framealpha=1)

def create_watch_results_tables(num_pars, no_nan=False):
    def _create_df(par_list, exp_name, modality):
        temp_df_list = []
        for par in par_list:
            temp_df = pd.DataFrame()
            try:
                temp_df[modality] = par.exp_modality_dict[exp_name][modality][modality]
                temp_df.reset_index(inplace=True, drop=True)
                par_num_col = par.data_fun.create_col(par.par_num, num_rows=temp_df.shape[0])
                temp_df.insert(loc=0, column="participant", value=par_num_col)
                block_col = par.exp_modality_dict[exp_name][modality]["block"]  
                temp_df.insert(loc=1, column="block", value=block_col)  
                trial_col = par.exp_modality_dict[exp_name][modality]["trial"]  
                temp_df.insert(loc=1, column="trial", value=trial_col)  
            except:
                temp_df[modality] = None
            temp_df_list.append(temp_df)
        df = pd.concat(temp_df_list, axis=0)
        df.reset_index(inplace=True, drop=True)
        return df

    def _data_to_excel(exp_name, data_dict, no_nan):
        if no_nan:
            filepath = os.path.join(os.getcwd(), "results", "watch_no_nan", f"{exp_name}_watch.xlsx")
        else:
            filepath = os.path.join(os.getcwd(), "results", "watch", f"{exp_name}_watch.xlsx")
            
        if not os.path.exists(os.path.dirname(filepath)):
            os.mkdir(os.path.dirname(filepath))
        
        with pd.ExcelWriter(filepath, engine='xlsxwriter') as writer:
            for modality, df in data_dict.items():
                df.to_excel(writer, sheet_name=modality, index=False)

    par_list = []                
    for i in range(num_pars):
        par_num = f"{(i+1):02d}"
        print("-----", par_num, "-----")
        par = Participant_Watch(par_num=par_num)
        par_list.append(par)

    for exp_name in par.exp_order:
        data_dict = {}
        for modality in par.modalities: 
            if modality == "ACC":
                temp_df_list = []
                for par in par_list:
                    temp_df = pd.DataFrame()
                    try:
                        temp_df["accel_x"] = par.exp_modality_dict[exp_name][modality]["accel_x"]
                        temp_df["accel_y"] = par.exp_modality_dict[exp_name][modality]["accel_y"]
                        temp_df["accel_z"] = par.exp_modality_dict[exp_name][modality]["accel_z"]
                        temp_df.reset_index(inplace=True, drop=True)
                        par_num_col = par.data_fun.create_col(par.par_num, num_rows=temp_df.shape[0])
                        temp_df.insert(loc=0, column="participant", value=par_num_col)
                        block_col = par.exp_modality_dict[exp_name][modality]["block"]
                        temp_df.insert(loc=1, column="block", value=block_col)
                        trial_col = par.exp_modality_dict[exp_name][modality]["trial"]  
                        temp_df.insert(loc=1, column="trial", value=trial_col)  
                    except:
                        temp_df["accel_x"] = None
                        temp_df["accel_y"] = None
                        temp_df["accel_z"] = None
                    temp_df_list.append(temp_df)
                ACC_df = pd.concat(temp_df_list, axis=0)
                ACC_df.reset_index(inplace=True, drop=True)
                if no_nan:
                    data_dict[modality] = ACC_df.dropna()
                else:
                    data_dict[modality] = ACC_df.dropna()
            elif modality == "BVP" or modality == "EDA" or modality == "HR" or modality == "IBI" or modality == "TEMP":
                modality_df = _create_df(par_list, exp_name, modality)
                if no_nan:
                    data_dict[modality] = modality_df.dropna()
                else:
                    data_dict[modality] = modality_df

        _data_to_excel(exp_name, data_dict, no_nan)

,match,stim_image.started,stim_text.started,stim_resp.corr,stim_resp.rt
0,0.0,44.218775,51.202135,0.0,0.790830
1,1.0,54.168875,61.152306,1.0,0.632662
2,1.0,64.169016,71.152435,1.0,0.856727
3,0.0,74.169165,81.152565,1.0,0.584455
4,1.0,84.169295,91.152706,1.0,0.632280
5,0.0,94.169400,101.152851,1.0,0.408110
7,0.0,109.469623,116.436360,1.0,0.596443
8,0.0,119.453147,126.453173,1.0,0.419742
9,0.0,129.469871,136.453275,1.0,0.563541
10,0.0,139.469983,146.453436,1.0,0.499348


In [82]:
# Example
par = Participant_Watch(par_num="01")
#print(par.exp_modality_dict["go_no_go"]["HR"])
#create_watch_results_tables(num_pars=15)

dict_values([      block  index    timestamps                   datetime   accel_x  \
0      None   1060  1.655319e+09 2022-06-15 14:43:17.156250  0.468750   
1      None   1061  1.655319e+09 2022-06-15 14:43:17.187500  0.468750   
2      None   1062  1.655319e+09 2022-06-15 14:43:17.218750  0.468750   
3      None   1063  1.655319e+09 2022-06-15 14:43:17.250000  0.468750   
4      None   1064  1.655319e+09 2022-06-15 14:43:17.281250  0.453125   
...     ...    ...           ...                        ...       ...   
14374  None  15434  1.655319e+09 2022-06-15 14:50:46.343750  0.562500   
14375  None  15435  1.655319e+09 2022-06-15 14:50:46.375000  0.546875   
14376  None  15436  1.655319e+09 2022-06-15 14:50:46.406250  0.546875   
14377  None  15437  1.655319e+09 2022-06-15 14:50:46.437500  0.546875   
14378  None  15438  1.655319e+09 2022-06-15 14:50:46.468750  0.546875   

        accel_y  accel_z  
0      0.015625   0.8750  
1      0.015625   0.8750  
2      0.015625   0.8750  
3 

c:\Users\zackg\OneDrive\Ayaz Lab\KernelFlow_Analysis\behav_analysis.py:168: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series([x]*num_rows)


dict_values([     block  index    timestamps                   datetime   accel_x  \
0     None  68610  1.655321e+09 2022-06-15 15:18:28.093750  0.640625   
1     None  68611  1.655321e+09 2022-06-15 15:18:28.125000  0.640625   
2     None  68612  1.655321e+09 2022-06-15 15:18:28.156250  0.640625   
3     None  68613  1.655321e+09 2022-06-15 15:18:28.187500  0.640625   
4     None  68614  1.655321e+09 2022-06-15 15:18:28.218750  0.656250   
...    ...    ...           ...                        ...       ...   
3345   NaN  71955  1.655321e+09 2022-06-15 15:20:12.625000  0.671875   
3346   NaN  71956  1.655321e+09 2022-06-15 15:20:12.656250  0.671875   
3347   NaN  71957  1.655321e+09 2022-06-15 15:20:12.687500  0.671875   
3348   NaN  71958  1.655321e+09 2022-06-15 15:20:12.718750  0.671875   
3349   NaN  71959  1.655321e+09 2022-06-15 15:20:12.750000  0.671875   

       accel_y   accel_z  
0     0.015625  0.765625  
1     0.015625  0.765625  
2     0.015625  0.765625  
3     0.015625

In [88]:
create_watch_results_tables(num_pars=1, no_nan=True)

----- 01 -----
dict_values([      trial block  index    timestamps                   datetime   accel_x  \
0      None  None   1060  1.655319e+09 2022-06-15 14:43:17.156250  0.468750   
1      None  None   1061  1.655319e+09 2022-06-15 14:43:17.187500  0.468750   
2      None  None   1062  1.655319e+09 2022-06-15 14:43:17.218750  0.468750   
3      None  None   1063  1.655319e+09 2022-06-15 14:43:17.250000  0.468750   
4      None  None   1064  1.655319e+09 2022-06-15 14:43:17.281250  0.453125   
...     ...   ...    ...           ...                        ...       ...   
14374  None  None  15434  1.655319e+09 2022-06-15 14:50:46.343750  0.562500   
14375  None  None  15435  1.655319e+09 2022-06-15 14:50:46.375000  0.546875   
14376  None  None  15436  1.655319e+09 2022-06-15 14:50:46.406250  0.546875   
14377  None  None  15437  1.655319e+09 2022-06-15 14:50:46.437500  0.546875   
14378  None  None  15438  1.655319e+09 2022-06-15 14:50:46.468750  0.546875   

        accel_y  accel_

c:\Users\zackg\OneDrive\Ayaz Lab\KernelFlow_Analysis\behav_analysis.py:168: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series([x]*num_rows)
c:\Users\zackg\OneDrive\Ayaz Lab\KernelFlow_Analysis\behav_analysis.py:168: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series([x]*num_rows)


dict_values([     trial block  index    timestamps                   datetime   accel_x  \
0     None  None  68610  1.655321e+09 2022-06-15 15:18:28.093750  0.640625   
1     None  None  68611  1.655321e+09 2022-06-15 15:18:28.125000  0.640625   
2     None  None  68612  1.655321e+09 2022-06-15 15:18:28.156250  0.640625   
3     None  None  68613  1.655321e+09 2022-06-15 15:18:28.187500  0.640625   
4     None  None  68614  1.655321e+09 2022-06-15 15:18:28.218750  0.656250   
...    ...   ...    ...           ...                        ...       ...   
3345   NaN   NaN  71955  1.655321e+09 2022-06-15 15:20:12.625000  0.671875   
3346   NaN   NaN  71956  1.655321e+09 2022-06-15 15:20:12.656250  0.671875   
3347   NaN   NaN  71957  1.655321e+09 2022-06-15 15:20:12.687500  0.671875   
3348   NaN   NaN  71958  1.655321e+09 2022-06-15 15:20:12.718750  0.671875   
3349   NaN   NaN  71959  1.655321e+09 2022-06-15 15:20:12.750000  0.671875   

       accel_y   accel_z  
0     0.015625  0.76562

In [6]:
dir(par.par_behav.go_no_go)
par.par_behav.by_block_ts_df["go_no_go"]

{(1655319109.6634579, 1655319216.3483195): 'go',
 (1655319227.9149013, 1655319363.4502645): 'GNG',
 (1655319368.2333953, 1655319484.4685206): 'go',
 (1655319490.3016665, 1655319653.8040273): 'GNG'}

In [ ]:
### Accelerometer ###
par.plot_modality("ACC")

In [ ]:
### Photoplethysmograph ###
par.plot_modality("BVP")

In [ ]:
### Electrodermal ###
par.plot_modality("EDA")

In [ ]:
### Heart Rate ###
par.plot_modality("HR")

In [ ]:
### Interbeat Interval ###
par.plot_modality("IBI")

In [ ]:
### Temperature ###
par.plot_modality("TEMP")